In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv(r'../data/train.csv')

# Pre-processing Data

In [4]:
#Check for Blank Values
data.isna().sum()

Unnamed: 0                             0
id                                     0
Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
satisfaction    

In [5]:
# Drop Blank Values
data.dropna(inplace=True)

In [6]:
# Drop columns which just show the serial number
data.drop(columns=['Unnamed: 0','id'], inplace=True)

In [7]:
data['satisfaction_labels'] = data['satisfaction'].apply(lambda x : 1 if x=='satisfied' else 0)

In [8]:
data.columns = data.columns.str.replace(' ', '_')

In [9]:
data.drop(columns=['satisfaction'], inplace=True)

# Data Splitting

In [10]:
train, val = train_test_split(data, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')

82875 train examples
20719 validation examples


# Data Conversion to TF Dataset Format

In [11]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('satisfaction_labels')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [12]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

In [13]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['Age'])
  print('A batch of targets:', label_batch )

Every feature: ['Gender', 'Customer_Type', 'Age', 'Type_of_Travel', 'Class', 'Flight_Distance', 'Inflight_wifi_service', 'Departure/Arrival_time_convenient', 'Ease_of_Online_booking', 'Gate_location', 'Food_and_drink', 'Online_boarding', 'Seat_comfort', 'Inflight_entertainment', 'On-board_service', 'Leg_room_service', 'Baggage_handling', 'Checkin_service', 'Inflight_service', 'Cleanliness', 'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes']
A batch of ages: tf.Tensor(
[11 26 44 49 44 20 37 32 34 55 43 77 57 19 18 35 38 36 58 49 46 42 54 25
 42 54 66 45 46 41 26 42], shape=(32,), dtype=int64)
A batch of targets: tf.Tensor([0 1 1 1 1 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0], shape=(32,), dtype=int64)


# Feature Transformation

In [14]:
element_spec = train_ds.element_spec

# Identify numerical and categorical columns
numerical_cols = []
categorical_cols = []
for key, value in element_spec[0].items():
    if value.dtype in (tf.float32, tf.float64, tf.int32, tf.int64):
        numerical_cols.append(key)
    else:
        categorical_cols.append(key)

print("Numerical columns:", numerical_cols)
print("Categorical columns:", categorical_cols)

Numerical columns: ['Age', 'Flight_Distance', 'Inflight_wifi_service', 'Departure/Arrival_time_convenient', 'Ease_of_Online_booking', 'Gate_location', 'Food_and_drink', 'Online_boarding', 'Seat_comfort', 'Inflight_entertainment', 'On-board_service', 'Leg_room_service', 'Baggage_handling', 'Checkin_service', 'Inflight_service', 'Cleanliness', 'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes']
Categorical columns: ['Gender', 'Customer_Type', 'Type_of_Travel', 'Class']


In [15]:
feature_columns = []

# numeric cols
for num_cols in numerical_cols:
  feature_columns.append(feature_column.numeric_column(num_cols))

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [16]:
for col_name in categorical_cols:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, data[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.
Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


# Model Building

In [17]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [18]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(256, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10


/Users/gorkem/miniconda3/envs/mlflow-env/lib/python3.11/site-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2590/2590 [==============================] - 3s 807us/step - loss: 1.1729 - accuracy: 0.7050 - val_loss: 0.4058 - val_accuracy: 0.8496
Epoch 2/10
2590/2590 [==============================] - 2s 767us/step - loss: 0.4410 - accuracy: 0.8111 - val_loss: 0.3537 - val_accuracy: 0.8523
Epoch 3/10
2590/2590 [==============================] - 2s 762us/step - loss: 0.3869 - accuracy: 0.8407 - val_loss: 0.3874 - val_accuracy: 0.8432
Epoch 4/10
2590/2590 [==============================] - 2s 763us/step - loss: 0.3574 - accuracy: 0.8567 - val_loss: 0.3757 - val_accuracy: 0.8478
Epoch 5/10
2590/2590 [==============================] - 2s 772us/step - loss: 0.3292 - accuracy: 0.8681 - val_loss: 0.2768 - val_accuracy: 0.8877
Epoch 6/10
2590/2590 [==============================] - 2s 752us/step - loss: 0.2862 - accuracy: 0.8841 - val_loss: 0.2431 - val_accuracy: 0.8974
Epoch 7/10
2590/2590 [==============================] - 2s 757us/step - loss: 0.2479 - accuracy: 0.8977 - val_loss: 0.2385 - val_accura

# Inference

In [21]:
# Reading Test Data
test_data = pd.read_csv(r'../data/test.csv')

test_data.drop(columns=['Unnamed: 0','id'], inplace=True)
test_data.columns = test_data.columns.str.replace(' ', '_')

test_data['satisfaction_bin_labels'] = test_data['satisfaction'].apply(lambda x : 1 if x=='satisfied' else 0)

test_features = {col: test_data[col].values for col in test_data.columns if col != 'satisfaction'}

# Make predictions on the test dataset
predictions = model.predict(test_features)

# Assuming predictions are probabilities, convert them to binary labels (0 or 1)
binary_predictions = np.round(predictions).astype(int)

# Print or use binary_predictions as needed
print("Binary Predictions:")
print(binary_predictions)

812/812 [==============================] - 0s 414us/step
Binary Predictions:
[[1]
 [1]
 [0]
 ...
 [0]
 [1]
 [0]]


In [22]:
# Evaluating Model on Test Data
from sklearn.metrics import classification_report

print(classification_report(test_data['satisfaction_bin_labels'].values, binary_predictions))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93     14573
           1       0.93      0.90      0.91     11403

    accuracy                           0.92     25976
   macro avg       0.92      0.92      0.92     25976
weighted avg       0.92      0.92      0.92     25976



# Save Model

In [24]:
model_filename = "dl_model_v1"

# Save the entire model (including architecture, optimizer, and learned weights)
model.save(f"../models/{model_filename}")

INFO:tensorflow:Assets written to: ../models/dl_model_v1/assets


INFO:tensorflow:Assets written to: ../models/dl_model_v1/assets


# Model Tuning

In [ ]:
# Below is a sample code for finetuning the model , you can customise it according to your implementation.
# We have provided some default parameters & layers which you can customize.

# def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    
#     model = tf.keras.models.Sequential()
#     model.add(tf.keras.kayers.InputLayer(input_shape=input_shape))
#     for layer in range(n_hidden):
#         model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
#     model.add(tf.keras.layers.Dense(1))
#     optimizer = tf.keras.optimizers.SGD(lr=learning_rate)
#     model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)
    
#     return model

# tune_model_obj = tf.keras.wrappers.scikit_learn.KerasClassifier(build_model)

# tune_model_obj.fit(X_train, y_train, epochs=10,
#                     validation_data=(X_valid, y_valid),
#                     callbacks=tf.keras.callbacks.EarlyStopping(patience=10))

# param_space = {...}

# rndm_search = RandomizedSearchCV(tune_model_obj, param_space, n_iter=10, cv=3)

# rndm_search.fit(X_train, y_train, epochs=10,
#                 validation_data=(X_valid, y_valid),
#                 callbacks=tf.keras.callbacks.EarlyStopping(patience=10))